In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [3]:
#Drop unnecessary columns
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
data.head()
#Encoding categorical data
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
#Concatenate the new columns with the original dataframe
data=pd.concat([data,geo_encoded_df],axis=1)
#Drop the original Geography column
data.drop(['Geography'],axis=1,inplace=True)
#Encoding categorical data
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0               1        101348.88       1               1.0   
1               1        112542.58       0               0.0   
2               0        113931.57       1               1.0   
3               0         93826.63       0               1.0   
4               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [4]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [5]:
## Devide the data into features and target variable
X=data.drop(['Exited'],axis=1)
y=data['Exited']

#Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Feature scaling
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [6]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [7]:
data

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619       0   42       2       0.00              1          1   
1             608       0   41       1   83807.86              1          0   
2             502       0   42       8  159660.80              3          1   
3             699       0   39       1       0.00              2          0   
4             850       0   43       2  125510.82              1          1   
...           ...     ...  ...     ...        ...            ...        ...   
9995          771       1   39       5       0.00              2          1   
9996          516       1   35      10   57369.61              1          1   
9997          709       0   36       7       0.00              1          0   
9998          772       1   42       3   75075.31              2          1   
9999          792       0   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0                  1        101348.88       1               1.0   
1                  1        112542.58       0               0.0   
2                  0        113931.57       1               1.0   
3                  0         93826.63       0               1.0   
4                  1         79084.10       0               0.0   
...              ...              ...     ...               ...   
9995               0         96270.64       0               1.0   
9996               1        101699.77       0               1.0   
9997               1         42085.58       1               1.0   
9998               0         92888.52       1               0.0   
9999               0         38190.78       0               1.0   

      Geography_Germany  Geography_Spain  
0                   0.0              0.0  
1                   0.0              1.0  
2                   0.0              0.0  
3                   0.0              0.0  
4                   0.0              1.0  
...                 ...              ...  
9995                0.0              0.0  
9996                0.0              0.0  
9997                0.0              0.0  
9998                1.0              0.0  
9999                0.0              0.0  

[10000 rows x 13 columns]

In [8]:
X_train.shape

(8000, 12)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

ModuleNotFoundError: No module named 'tensorflow.python.distribute.load_context'

In [ ]:
## Building the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),## HL1 connected to input layer
    Dense(32,activation='relu'),## HL2
    Dense(1,activation='sigmoid')## Output layer
])

In [ ]:
model.summary()

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
## Compiling the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
## Setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir='logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
## setup early stopping
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
### Training the model
history=model.fit(X_train,y_train,epochs=100,batch_size=32,validation_split=0.2,callbacks=[tensorboard_callback,early_stopping])
## Evaluating the model
loss,accuracy=model.evaluate(X_test,y_test)

In [ ]:
model.save('model.h5')

In [ ]:
##Load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit